# Analysing Face Detection on African data

## Importing Libraries

In [11]:
import gc
import os
import sys
import math
import glob
import random
import numpy as np
from tqdm import tqdm
from time import sleep

In [9]:
import pandas

In [10]:
import cv2
import dlib
from imutils import face_utils
from skimage.feature import hog
from skimage import data,exposure

In [8]:
import matplotlib 
%matplotlib inline
import matplotlib.pyplot as plt

## Defining paths

#### Path to data folder

In [15]:
path_folder_bagamoyo_data = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/'

In [ ]:
#### Path t

In [16]:
path_folder_all_frames = path_folder_bagamoyo_data + '/bagamoyo_frames_all_in_one/'

In [17]:
path_folder_wise_frames = path_folder_bagamoyo_data + '/bagamoyo_frames_folder_wise/'